In [ ]:
!nvidia-smi

Tue May 26 18:09:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
# %%writefile setup.sh

# !git clone https://github.com/NVIDIA/apex
# !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
# !sh setup.sh

Writing setup.sh


In [ ]:
# !pip3 install -U sentence-transformers
# import pandas as pd
# import numpy as np
# from sentence_transformers import SentenceTransformer
# from scipy import spatial
# import time

embedder = SentenceTransformer('bert-base-nli-mean-tokens')

train_df = pd.read_csv("dev.tsv", delimiter="\t", names=["q1", "q2", "label"], dtype="object")

train_q1 = train_df["q1"]
stime = time.time()
train_q1_emb = embedder.encode(train_q1)
print(time.time()-stime)

train_q2 = train_df["q2"]
stime = time.time()
train_q2_emb = embedder.encode(train_q2)
print(time.time()-stime)

corpussize = len(train_q1)
sims =  [1 - spatial.distance.cosine(train_q1_emb[i], train_q2_emb[i]) for i in range(corpussize)]



95.26211404800415
94.88235187530518


In [ ]:
for i in range(10000):
    if(int(labels[i])==1):
        print(train_q1[i])
        print(train_q2[i])
        print(sims[i])
        print(labels[i])

Streaming output truncated to the last 5000 lines.
how do i wake up earlier ? 
0.9530764818191528
1
what is the best answer when we are asked by interviewers why should we hire you ? 
what is the best answer for why i should hire you ? 
0.9111599922180176
1
what can you take to lose weight fast ? 
is there a technique to lose weight fast ? 
0.9391511082649231
1
what is your review of the 2016 macbook pro ? 
is macbook pro 2016 worth it ? 
0.8657593727111816
1
my uber account has been disabled . can it enabled ? 
why has my uber account been disabled and how do i reactivate it ? 
0.9022411108016968
1
which are the top ten best hollywood romantic movies ? 
what are some of the best romantic hollywood movies ? 
0.7737889885902405
1
what is the difference between chinese and american education ? 
what is the difference between chinese education and american education ? 
0.9894655346870422
1
what is tangent universe ? 
what is the tangent universe ? 
0.9827089905738831
1
should india declar

In [ ]:
thresh = 0.7
while(thresh<1):
    res = [1 if x>=thresh else 0 for x in sims]
    labels = train_df["label"]
    compare = [1 if res[i]==int(labels[i]) else 0 for i in range(corpussize)]
    # print(compare)
    print("thresh:",thresh, ", acc:", sum(compare)/float(len(compare)))
    thresh += 0.05

thresh: 0.7 , acc: 0.6246810829555869
thresh: 0.75 , acc: 0.6685492061132992
thresh: 0.8 , acc: 0.7043422258551931
thresh: 0.8500000000000001 , acc: 0.728270293032127
thresh: 0.9000000000000001 , acc: 0.7285427658467712
thresh: 0.9500000000000002 , acc: 0.6978524188154863


In [ ]:
# !pip install simpletransformers

In [ ]:
import simpletransformers
from simpletransformers.classification import ClassificationModel
import pandas as pd
import sklearn
import logging
import time


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)


In [ ]:
train_df = pd.read_csv("train.tsv", delimiter="\t", names=["text_a", "text_b", "labels"], dtype="object")
train_df["labels"] = train_df["labels"].astype(int)

train_set = train_df

eval_set = pd.read_csv("dev.tsv", delimiter="\t", names=["text_a", "text_b", "labels"], dtype="object")
eval_set["labels"] = eval_set["labels"].astype(int)

In [ ]:

train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 3,
    # 'evaluate_during_training' : True,
    'train_batch_size': 32,
    'eval_batch_size': 32,
    'save_eval_checkpoints' : False,
    'save_model_every_epoch' : False,
    'use_cached_eval_features' : True,
    'tensorboard_dir' : "/content/outputs/",
    # "evaluate_during_training_verbose": True,
}
# # Create a ClassificationModel
model = ClassificationModel('albert', 'albert-base-v2', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)

stime = time.time()
# # Train the model
model.train_model(train_set, eval_df=eval_set)

# # Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_set, acc=sklearn.metrics.accuracy_score)
print(time.time()-stime)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.703344

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.709094Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.668755

...

Running loss: 0.440890Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.864806Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.618165

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.369888Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.217211Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 0.259577Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 0.335698Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.560314Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Running loss: 0.046536


Running loss: 0.437501Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.179793Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.038637


Running loss: 0.159471Buffered data was truncated after reaching the output size limit.

In [ ]:
test_set = train_df[330000:]
test_set.reset_index(drop=True, inplace=True)
actual = test_set["labels"].tolist()
test_set.drop(columns="labels", inplace=True)

predictions, raw_outputs = model.predict(test_set.values.tolist())
# print(predictions)
# print(actual)

compare = [1 if predictions[i]==actual[i] else 0 for i in range(len(actual))]
# print(compare)
print(sum(compare)/float(len(compare)))
# print(raw_outputs)